In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## 导入必要的库

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from colorama import Fore, Style, init

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator,TransformerMixin

from sklearn.feature_selection import mutual_info_classif
from scipy import stats

from sklearn.decomposition import PCA

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV

## 读取数据

In [ ]:
train_df = pd.read_csv('/kaggle/input/playground-series-s3e22/train.csv', index_col = 'id')
test_df = pd.read_csv('/kaggle/input/playground-series-s3e22/test.csv', index_col = 'id')
original_df = pd.read_csv('/kaggle/input/horse-survival-dataset/horse.csv')
train_df.index.name = None
test_df.index.name = None

original_df.index = range(len(original_df))
original_df.index += max(test_df.index) + 1
original_df = original_df.reindex(train_df.columns, axis=1)

# 融合的数据
trainANDoriginal_df = pd.concat([train_df, original_df], axis=0, ignore_index = True)

feature = train_df.columns[:-1]
target = train_df.columns[-1]
target_map = {"lived":1,"died":2,"euthanized":3}
train_df[target] = train_df[target].map(target_map).astype(np.int8)
original_df[target] = original_df[target].map(target_map).astype(np.int8)
trainANDoriginal_df[target] = trainANDoriginal_df[target].map(target_map).astype(np.int8)

## datadict

In [ ]:
# with open('/kaggle/input/horse-colic/datadict.txt', 'r') as file:
#    for line in file:
#        print(line)

## 常用函数

In [ ]:
def PrintColor(text:str, color = Fore.BLUE, style = Style.BRIGHT):
    print(style + color + text + style)

# 字典编码函数  排序映射赋值（有序离散object处理）
def change_object_cols(se):
    value = se.unique().tolist()
    value.sort()
    return se.map(pd.Series(range(len(value),index=value))).values
# 多变量分析  将两个变量融合进而分析
def combine_feature(df):
    cols = df.columns
    feature1 = df[cols[0]].astype(str).values.tolist()
    feature2 = df[cols[1]].astype(str).values.tolist()
    return pd.Series([feature1[i] + '&' + feature2[i] for i in range(df.shape[0])])

## 对抗验证 adversarial CV

In [ ]:
# # Performing adversarial CV between the 2 specified datasets:-
# def Do_AdvCV(df1:pd.DataFrame, df2:pd.DataFrame, source1:str, source2:str):
#     "This function performs an adversarial CV between the 2 provided datasets if needed by the user";
    
#     # Adversarial CV per column:-
#     ftre = pp.test.select_dtypes(include = np.number).\
#     drop(columns = ['id', "Source"], errors = 'ignore').columns;
#     adv_cv = {};

#     for col in ftre:
#         shuffle_state = np.random.randint(low = 10, high = 100, size= 1);

#         full_df = \
#         pd.concat([df1[[col]].assign(Source = source1), df2[[col]].assign(Source = source2)], 
#                   axis=0, ignore_index = True).\
#         sample(frac = 1.00, random_state = shuffle_state);

#         full_df = full_df.assign(Source_Nb = full_df['Source'].eq(source2).astype(np.int8));

#         # Checking for adversarial CV:-
#         model = LGBMClassifier(random_state = CFG.state, max_depth = 6, learning_rate = 0.05);
#         cv    = all_cv['SKF'];
#         score = np.mean(cross_val_score(model, 
#                                         full_df[[col]], 
#                                         full_df.Source_Nb, 
#                                         scoring= 'roc_auc', 
#                                         cv     = cv)
#                        );
#         adv_cv.update({col: round(score, 4)});
#         collect();
    
#     del ftre;
#     collect();
    
#     fig, ax = plt.subplots(1,1,figsize = (12, 5));
#     pd.Series(adv_cv).plot.bar(color = 'tab:blue', ax = ax);
#     ax.axhline(y = 0.60, color = 'red', linewidth = 2.75);
#     ax.grid(**CFG.grid_specs); 
#     plt.yticks(np.arange(0.0, 0.81, 0.05));
#     plt.show();
    
# # Implementing the adversarial CV:-
# if CFG.adv_cv_req == "Y":
#     PrintColor(f"\n---------- Adversarial CV - Train vs Original ----------\n", 
#                color = Fore.MAGENTA);
#     Do_AdvCV(df1 = pp.train, df2 = pp.original, source1 = 'Train', source2 = 'Original');
    
#     PrintColor(f"\n---------- Adversarial CV - Train vs Test ----------\n", 
#                color = Fore.MAGENTA);
#     Do_AdvCV(df1 = pp.train, df2 = pp.test, source1 = 'Train', source2 = 'Test');
    
#     PrintColor(f"\n---------- Adversarial CV - Original vs Test ----------\n", 
#                color = Fore.MAGENTA);
#     Do_AdvCV(df1 = pp.original, df2 = pp.test, source1 = 'Original', source2 = 'Test');   
    
# if CFG.adv_cv_req == "N":
#     PrintColor(f"\nAdversarial CV is not needed\n", color = Fore.RED);
    
# collect();
# print();

## EDA

In [ ]:
# Train
PrintColor(f"\nTrain set head", color = Fore.BLUE)
display(train_df.head(5).style.format(precision = 3))
PrintColor(f"\nTrain set description", color = Fore.BLUE)
display(train_df.describe(percentiles= [0.05, 0.25, 0.50, 0.75, 0.9, 0.95, 0.99]).transpose().
                            drop(columns = ['count'], errors = 'ignore').
                            style.format(formatter = '{:,.2f}').
                            background_gradient(cmap = 'Purples'))
PrintColor(f"\nTrain set Information", color = Fore.BLUE)
display(train_df.info())

# Original
PrintColor(f"\nOriginal set head", color = Fore.BLUE)
display(original_df.head(5).style.format(precision = 3))
PrintColor(f"\nOriginal set description", color = Fore.BLUE)
display(original_df.describe(percentiles= [0.05, 0.25, 0.50, 0.75, 0.9, 0.95, 0.99]).transpose().
                            drop(columns = ['count'], errors = 'ignore').
                            style.format(formatter = '{:,.2f}').
                            background_gradient(cmap = 'Purples'))
PrintColor(f"\nOriginal set Information", color = Fore.BLUE)
display(original_df.info())

# TrainANDoriginal
PrintColor(f"\nTrainANDoriginal set head", color = Fore.BLUE)
display(trainANDoriginal_df.head(5).style.format(precision = 3))
PrintColor(f"\nTrainANDoriginal set description", color = Fore.BLUE)
display(trainANDoriginal_df.describe(percentiles= [0.05, 0.25, 0.50, 0.75, 0.9, 0.95, 0.99]).transpose().
                            drop(columns = ['count'], errors = 'ignore').
                            style.format(formatter = '{:,.2f}').
                            background_gradient(cmap = 'Purples'))
PrintColor(f"\nTrainANDoriginal set Information", color = Fore.BLUE)
display(trainANDoriginal_df.info())

# Test
PrintColor(f"\nTest set head", color = Fore.BLUE)
display(test_df.head(5).style.format(precision = 3))
PrintColor(f"\nTest set description", color = Fore.BLUE)
display(test_df.describe(percentiles= [0.05, 0.25, 0.50, 0.75, 0.9, 0.95, 0.99]).transpose().
                            drop(columns = ['count'], errors = 'ignore').
                            style.format(formatter = '{:,.2f}').
                            background_gradient(cmap = 'Purples'))
PrintColor(f"\nTest set Information", color = Fore.BLUE)
display(test_df.info())

## 1.hospital_number 有重复、出现数值很大的编号、是否应该当成离散量重新编一下码----作为类别变量----独热编码
##   hospital_number 在test中出现但在train中没出现 ---- 需要test和train合并编码
## 2.lesion_1、lesion_2、lesion_3数据比较有问题，需要如何处理
##  2.1lesion_2为非0，outcome为非died
##     可处理df['lesion_2'] = df['lesion_2'].apply(lambda x:1 if x>0 else 0)  （预测died变好？）
##     作为类别变量处理
##  2.2lesion_3只有两个值，lession_3非0，outcome非died
##     同上处理？
##     作为类别变量处理
##  2.3查看到其特征解释貌似可以拆解开分为多个特征
##     见datadict

## other tricks
## 1.马儿的温度影响outcome，偏离37.8越多越容易死
##   df["deviation_from_normal_temp"] = df["rectal_temp"].apply(lambda x: abs(x - 37.8))
## 2.pulse, respiatory_rate, packed_cell_volume是否可以如1

In [ ]:
# 数据正确性验证
# 缺失值
# 异常值
# 规律性分析 （单变量分析、多变量分析）
# 离散型变量区分、名义型变量（男女-10）、有序（考虑是否当成连续）

In [ ]:
# 缺失数据分析1
PrintColor(f"\nTrain set null values", color = Fore.BLUE)
train_missing = train_df.isnull().sum() * 100 / len(train_df)
train_missing = train_missing[train_missing != 0]
train_missing = pd.DataFrame({'missing percent': train_missing})
train_missing = train_missing.sort_values('missing percent', ascending=False)
display(train_missing)
for col in train_missing.index:
    print(col,train_df[col].unique())#此处缺失值如何去补充
    
PrintColor(f"\nOriginal set null values", color = Fore.BLUE)
original_missing = original_df.isnull().sum() * 100 / len(original_df)
original_missing = original_missing[original_missing != 0]
original_missing = pd.DataFrame({'missing percent': original_missing})
original_missing = original_missing.sort_values('missing percent', ascending=False)
display(original_missing)
for col in original_missing.index:
    print(col,original_df[col].unique())#此处缺失值如何去补充
    
PrintColor(f"\nTrainAndOriginal set null values", color = Fore.BLUE)
trainANDoriginal_missing = trainANDoriginal_df.isnull().sum() * 100 / len(trainANDoriginal_df)
trainANDoriginal_missing = trainANDoriginal_missing[trainANDoriginal_missing != 0]
trainANDoriginal_missing = pd.DataFrame({'missing percent': trainANDoriginal_missing})
trainANDoriginal_missing = trainANDoriginal_missing.sort_values('missing percent', ascending=False)
display(trainANDoriginal_missing)
for col in trainANDoriginal_missing.index:
    print(col,trainANDoriginal_df[col].unique())#此处缺失值如何去补充
    
PrintColor(f"\nTest set null values", color = Fore.BLUE)
test_missing = test_df.isnull().sum() * 100 / len(test_df)
test_missing = test_missing[test_missing != 0]
test_missing = pd.DataFrame({'missing percent': test_missing})
test_missing = test_missing.sort_values('missing percent', ascending=False)
display(test_missing)
for col in test_missing.index:
    print(col,test_df[col].unique())#此处缺失值如何去补充

In [ ]:
# 缺失数据分析2
def summary_df(train_df,original_df,trainANDoriginal_df,test_df):
    summary = pd.DataFrame(train_df.dtypes, columns=['dtypes'])
    summary['train_missing#'] = train_df.isna().sum()
    summary['train_missing%'] = (train_df.isna().sum())/len(train_df)
    summary['train_uniques'] = train_df.nunique().values
    summary['train_count'] = train_df.count().values
    # summary['train_skew'] = train_df.skew().values
    summary['original_missing#'] = original_df.isna().sum()
    summary['original_missing%'] = (original_df.isna().sum())/len(original_df)
    summary['original_uniques'] = original_df.nunique().values
    summary['original_count'] = original_df.count().values
    # summary['original_skew'] = original_df.skew().values
    summary['test_missing#'] = test_df.isna().sum()
    summary['test_missing%'] = (test_df.isna().sum())/len(test_df)
    summary['test_uniques'] = test_df.nunique().values
    summary['test_count'] = test_df.count().values
    # summary['test_skew'] = train_df.skew().values
    summary['trainANDoriginal_missing#'] = trainANDoriginal_df.isna().sum()
    summary['trainANDoriginal_missing%'] = (trainANDoriginal_df.isna().sum())/len(trainANDoriginal_df)
    summary['trainANDoriginal_uniques'] = trainANDoriginal_df.nunique().values
    summary['trainANDoriginal_count'] = trainANDoriginal_df.count().values
    # summary['trainANDoriginal_skew'] = trainANDoriginal_df.skew().values
    return summary
summary_df(train_df[feature],original_df[feature],trainANDoriginal_df[feature],test_df[feature]).style.background_gradient(cmap='Blues')

## 1.缺失值数据均为object类型
## 2.缺失值数据中的nan，有些还有none的存在，应该如何处理nan。  检查了一下数据发现是大写的None和小写的none的问题？
##   好像两者是不同的
## 3.

In [ ]:
# 寻找object类型、或者直接规定
dtypes = train_df.dtypes.to_dict()
cat_cols = []
num_cols = []
for column, typ in dtypes.items():
    if typ == "object":
        cat_cols.append(column)
    else:
        num_cols.append(column)

# 独特性验证(类别类、独特类（id等）)
for col in cat_cols:
    print('Train   ',col,train_df[col].unique())
    print('Original   ',col,original_df[col].unique())
    if col==target:
        continue
    print('Test   ',col,test_df[col].unique())

def unique_validation(train_df,test_df,original_df,trainANDoriginal_df,features):
    for feature in features:
        print('train',feature,train_df[feature].unique())
        print('test',feature,test_df[feature].unique())
        print('original',feature,original_df[feature].unique())
        print('trainANDoriginal',feature,trainANDoriginal_df[feature].unique())

        print('original_unique_to_train',feature,[original_unique_to_train for original_unique_to_train in original_df[feature].unique() if original_unique_to_train not in train_df[feature].unique()])
        print('test_unique_to_train',feature,[test_unique_to_train for test_unique_to_train in test_df[feature].unique() if test_unique_to_train not in train_df[feature].unique()])
        print('train_unique_to_original',feature,[train_unique_to_original for train_unique_to_original in train_df[feature].unique() if train_unique_to_original not in original_df[feature].unique()])
        print('train_unique_to_test',feature,[train_unique_to_test for train_unique_to_test in train_df[feature].unique() if train_unique_to_test not in test_df[feature].unique()])
        print('original_unique_to_test',feature,[original_unique_to_test for original_unique_to_test in original_df[feature].unique() if original_unique_to_test not in test_df[feature].unique()])
        print('test_unique_to_original',feature,[test_unique_to_original for test_unique_to_original in test_df[feature].unique() if test_unique_to_original not in original_df[feature].unique()])

        print('trainANDoriginal_unique_to_test',feature,[trainANDoriginal_unique_to_test for trainANDoriginal_unique_to_test in trainANDoriginal_df[feature].unique() if trainANDoriginal_unique_to_test not in test_df[feature].unique()])
        print('test_unique_to_trainANDoriginal',feature,[test_unique_to_trainANDoriginal for test_unique_to_trainANDoriginal in test_df[feature].unique() if test_unique_to_trainANDoriginal not in trainANDoriginal_df[feature].unique()])

unique_validation(train_df,test_df,original_df,trainANDoriginal_df,['lesion_1','lesion_2','lesion_3'])#,'hospital_number'

In [ ]:
#单变量分布1
fig, axes = plt.subplots(len(cat_cols), 4, figsize = (20, len(cat_cols)* 2), 
                             gridspec_kw = {'wspace': 0.35, 'hspace': 0.5})
for i, column in enumerate(cat_cols):
    ax1 = axes[i, 0]
    ax2 = axes[i, 1]
    ax3 = axes[i, 2]
    ax4 = axes[i, 3]
    a = train_df[column].value_counts(normalize = True)
    a.sort_index().plot.barh(ax = ax1, color = 'purple')
    ax1.set_title(f"{column}_Train")
    ax1.set(ylabel = '')
    b = original_df[column].value_counts(normalize = True)
    b.sort_index().plot.barh(ax = ax2, color = 'purple')
    ax2.set_title(f"{column}_Original")
    ax2.set(ylabel = '')
    d = trainANDoriginal_df[column].value_counts(normalize = True)
    d.sort_index().plot.barh(ax = ax4, color = 'purple')
    ax4.set_title(f"{column}_TrainAndOriginal")
    ax4.set(ylabel = '')
    if column==target:
        continue
    c = test_df[column].value_counts(normalize = True)
    c.sort_index().plot.barh(ax = ax3, color = 'purple')
    ax3.set_title(f"{column}_Test")
    ax3.set(ylabel = '')

## 1.pain  训练集slight  测试集moderate
##   考虑两者合并之后编码
## 2.peristalsis 训练集多一个distend_small（1个样本）
## 3.nasogastric_reflux 训练集多一个slight（1个样本）
## 4.rectal_exam_feces 训练集多一个serosanguious（1个样本）
##   234不删除可能也不影响，删除好像也不影响
## 5.mucous 中的pale_cyanotic分布不太一样（异常突出）
## 6.abdominal_distention、nasogastric_tube、nasogastric_reflux中的none   
##   考虑将None和none 都做none（普通字符串none的意义）处理？
##  （用isnull统计的None被视为无值，但是none被视为普通字符串）
##   此处处理完之后还需要做一下训练集和测试集分布分析


In [ ]:
# 单变量分布
fig, axes = plt.subplots(len(num_cols), 5, figsize = (20, len(num_cols)* 4), 
                             gridspec_kw = {'wspace': 0.35, 'hspace': 0.5, 'width_ratios': [0.80,0.20, 0.20, 0.20, 0.20]})
for i, column in enumerate(num_cols):
    ax = axes[i,0]
    a = train_df[column]
    a.plot(ax = ax,kind='density', color = 'purple',label='Train')
    b = original_df[column]
    b.plot(ax = ax,kind='density', color = 'pink',label='Original')
    if column not in ['lesion_3','outcome']:
        c = test_df[column]
        c.plot(ax = ax,kind='density', color = 'red',label='Test')
    d = trainANDoriginal_df[column]
    d.plot(ax = ax,kind='density', color = 'green',label='TrAndOr')
    ax.set_title(f"{column}_Train_vs_Test_vs_Original_vs_TrAndOr")
    ax.set(ylabel = '')
    ax.legend(['Train','Original','Test','TrAndOr'])
     
    
    ax = axes[i,1]
    sns.boxplot(y = a, width = 0.25,saturation = 0.90, linewidth = 0.90, fliersize= 2.25, color = '#037d97',ax = ax)
    ax.set(xlabel = '', ylabel = '')
    ax.set_title(f"Train",fontsize = 9, fontweight= 'bold')
    ax = axes[i,2]
    sns.boxplot(y = b, width = 0.25, fliersize= 2.25,saturation = 0.6, linewidth = 0.90, color = '#E4591E',ax = ax) 
    ax.set(xlabel = '', ylabel = '')
    ax.set_title(f"Original",fontsize = 9, fontweight= 'bold')
    ax = axes[i,4]
    sns.boxplot(y = d, width = 0.25, fliersize= 2.25,saturation = 0.6, linewidth = 0.90, color = '#B78C25',ax = ax) 
    ax.set(xlabel = '', ylabel = '')
    ax.set_title(f"TrainAndOriginal",fontsize = 9, fontweight= 'bold')
    if column not in ['lesion_3','outcome']:
        ax = axes[i,3]
        sns.boxplot(y = c, width = 0.25, fliersize= 2.25,saturation = 0.6, linewidth = 0.90, color = '#544E91',ax = ax)
        ax.set(xlabel = '', ylabel = '')
        ax.set_title(f"Test",fontsize = 9, fontweight= 'bold')
# 多变量分析

## 1.单变量分析，训练集和测试集分布基本一致，除了hospital_number和lesion_3差别
## 2.hospital_number
## 3.lesion_3

In [ ]:
# 联合outcome的单变量分布
def plot_count(train_df,original_df,trainANDoriginal_df,columns,hue,other_features=['lesion_1','lesion_2','lesion_3','hospital_number']):
    columns = columns + other_features
    n_rows = len(columns)
    fig, axes = plt.subplots(n_rows, 3, figsize=(17, 4 * n_rows))
    for i, column in enumerate(columns):
        ax = axes[i,0]
        sns.countplot(data=train_df, x=column, ax=ax,hue=hue)
        ax.set_title(f'Tarin_{column}_Counts', fontsize=18)
        ax.set_xlabel(None, fontsize=16)
        ax.set_ylabel(None, fontsize=16)
        ax.tick_params(axis='x', rotation=10)
        for p in ax.patches:
            value = int(p.get_height())
            ax.annotate(f'{value:.0f}', (p.get_x() + p.get_width() / 2, p.get_height()),
                           ha='center', va='bottom', fontsize=9)
            
        ax = axes[i,1]
        sns.countplot(data=original_df, x=column, ax=ax,hue=hue)
        ax.set_title(f'Original_{column}_Counts', fontsize=18)
        ax.set_xlabel(None, fontsize=16)
        ax.set_ylabel(None, fontsize=16)
        ax.tick_params(axis='x', rotation=10)
        for p in ax.patches:
            value = int(p.get_height())
            ax.annotate(f'{value:.0f}', (p.get_x() + p.get_width() / 2, p.get_height()),
                           ha='center', va='bottom', fontsize=9)
            
        ax = axes[i,2]
        sns.countplot(data=trainANDoriginal_df, x=column, ax=ax,hue=hue)
        ax.set_title(f'TarinAndOriginal_{column}_Counts', fontsize=18)
        ax.set_xlabel(None, fontsize=16)
        ax.set_ylabel(None, fontsize=16)
        ax.tick_params(axis='x', rotation=10)
        for p in ax.patches:
            value = int(p.get_height())
            ax.annotate(f'{value:.0f}', (p.get_x() + p.get_width() / 2, p.get_height()),
                           ha='center', va='bottom', fontsize=9)

    ylim_top = ax.get_ylim()[1]
    ax.set_ylim(top=ylim_top * 1.1)
    for i in range(len(columns), len(axes)):
        axes[i,0].axis('off')
        axes[i,1].axis('off')
        axes[i,2].axis('off')

    # fig.suptitle(plotname, fontsize=25, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
plot_count(train_df,original_df,trainANDoriginal_df,[cat_col for cat_col in cat_cols if cat_col != target],target,other_features=['lesion_2','lesion_3'])

In [ ]:
# 双变量配对分析
def plot_pair(train_df,num_var,target,plotname):
    g = sns.pairplot(data=train_df, x_vars=num_var, y_vars=num_var, hue=target, corner=True)
    g._legend.set_bbox_to_anchor((0.8, 0.7))
    g._legend.set_title(target)
    g._legend.loc = 'upper center'
    g._legend.get_title().set_fontsize(14)
    for item in g._legend.get_texts():
        item.set_fontsize(14)

    plt.suptitle(plotname, ha='center', fontweight='bold', fontsize=25, y=0.98)
    plt.show()

# plot_pair(train_df,num_cols,target,plotname = 'Scatter Matrix with Target Of Train')
# plt.tight_layout()
# plot_pair(original_df,num_cols,target,plotname = 'Scatter Matrix with Target Of Original')
# plt.tight_layout()
# plot_pair(trainANDoriginal_df,num_cols,target,plotname = 'Scatter Matrix with Target Of TrainAndOriginal')
# plt.tight_layout()

In [ ]:
method = "pearson"
plt.figure(figsize=(15,15))
corr = train_df[num_cols].corr(method = method)# 皮尔逊相关
mask = np.triu(np.triu(corr))
sns.heatmap(data = corr,annot=True, cmap = 'Blues', mask=mask)
plt.title(method+"_corr_of_train")

plt.figure(figsize=(15,15))
corr = original_df[num_cols].corr(method = method)# 皮尔逊相关
mask = np.triu(np.triu(corr))
sns.heatmap(data = corr,annot=True, cmap = 'Blues', mask=mask)
plt.title(method+"_corr_of_original")

plt.figure(figsize=(15,15))
corr = test_df[[num_col for num_col in num_cols if num_col != target]].corr(method = "pearson")# 皮尔逊相关
mask = np.triu(np.triu(corr))
sns.heatmap(data = corr,annot=True, cmap = 'Blues', mask=mask)
plt.title(method+"_corr_of_test")

plt.figure(figsize=(15,15))
corr = trainANDoriginal_df[num_cols].corr(method = method)# 皮尔逊相关
mask = np.triu(np.triu(corr))
sns.heatmap(data = corr,annot=True, cmap = 'Blues', mask=mask)
plt.title(method+"_corr_of_trainANDoriginal")

## 1.pulse 和 respiratory_rate、packed_cell_volume正相关比较明显
## 2.nasogastric_reflux_ph 和total_protein、abdomo_protein分别呈负相关、正相关
## 3.total_protein和abdomo_protein呈正相关
## 4.lesion_2和lesion_3呈正相关

## 卡方检验

In [ ]:
def chi_squared_test(df, input_var, target_var, significance_level=0.05):
    contingency_table = pd.crosstab(df[input_var], df[target_var])
    chi2, p, _, _ = stats.chi2_contingency(contingency_table)
    
    if p < significance_level:
        print(f'\033[32m{input_var} has a significant relationship with the target variable.\033[0m') 
    else:
        print(f'\033[31m{input_var} does not have a significant relationship with the target variable.\033[0m')  

for cat_col in cat_cols:
    chi_squared_test(train_df, cat_col, target)
    chi_squared_test(original_df, cat_col, target)
    chi_squared_test(trainANDoriginal_df, cat_col, target)

## **特征工程**

In [ ]:
## 1.通用组合特征
## 2.业务统计特征
## 我们需要融合？
## NLP特征池衍生（from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer）

In [ ]:
print(num_cols,f'\n',cat_cols)

## **pipeline**

In [ ]:
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy="median")),
#     ('std_scaler', StandardScaler()),
])
cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy="constant", fill_value="NaN")),
    ('encoder', OrdinalEncoder())
])
full_pipeline = ColumnTransformer([
    ("num", num_pipeline, num_cols),
    ("cat", cat_pipeline, cat_cols),
])
train_prepared = full_pipeline.fit_transform(train_df)
display(train_df)
display(pd.DataFrame(train_prepared))

# 或者单独处理？
# Scaling:-
# class Scaler(TransformerMixin, BaseEstimator):
#     """
#     This class aims to create scaling for the provided dataset
#     """;
    
#     def __init__(self, scl_method: str, scale_req: str, scl_cols):
#         self.scl_method = scl_method;
#         self.scale_req  = scale_req;
#         self.scl_cols   = scl_cols;
        
#     def fit(self,X, y=None, **params):
#         "This function calculates the train-set parameters for scaling";
        
#         self.params          = X[self.scl_cols].describe(percentiles = [0.25, 0.50, 0.75]).drop(['count'], axis=0).T;
#         self.params['iqr']   = self.params['75%'] - self.params['25%'];
#         self.params['range'] = self.params['max'] - self.params['min'];
        
#         return self;
    
#     def transform(self,X, y=None, **params):  
#         "This function transform the relevant scaling columns";
        
#         df = X.copy();
#         if self.scale_req == "Y":
#             if CFG.scl_method == "Z":
#                 df[self.scl_cols] = (df[self.scl_cols].values - self.params['mean'].values) / self.params['std'].values;
#             elif CFG.scl_method == "Robust":
#                 df[self.scl_cols] = (df[self.scl_cols].values - self.params['50%'].values) / self.params['iqr'].values;
#             elif CFG.scl_method == "MinMax":
#                 df[self.scl_cols] = (df[self.scl_cols].values - self.params['min'].values) / self.params['range'].values;
#         else:
#             PrintColor(f"Scaling is not needed", color = Fore.RED);
    
#         return df;

In [ ]:
# class Xformer(TransformerMixin, BaseEstimator):
#     def __init__(self, hnb_only_test:list, hnb_only_train: list): 
#         self.sec_ftre_req = CFG.sec_ftre_req;
#         self.hnb_only_test = hnb_only_test;
#         self.hnb_only_train = hnb_only_train;
    
#     def fit(self, X, y= None, **params):
#         hnb_diff = \
#         {nb: abs(nb - base_val) for nb in X['hospital_number'].unique() for base_val in self.hnb_only_test};
#         self.hnb_mapper = {self.hnb_only_test[0]: min(hnb_diff, key= hnb_diff.get)}
    
#         self.ip_cols = X.columns;
#         return self;
    
#     def transform(self, X, y= None, **params):       
#         global strt_ftre;
#         df    = X.copy();  
        
#         # Correcting the test-only hospital number with the nearest training set number:-
#         try:
#             df['hospital_number'] = df['hospital_number'].astype(np.int64).map(self.hnb_mapper).astype(np.int64);
#         except:
#             PrintColor(f"---> Check the hospital number assignment", Fore.RED);
#             df.loc[df['hospital_number'] == 528338, "hospital_number"] = 528355;
        
#         # Splitting the lesion 1 feature into its components:-
#         l1_prf = \
#         df['lesion_1'].astype(np.int64).\
#         astype(str).str.split('', expand = True).\
#         applymap(lambda x: 0 if x in ['', None, np.NaN] else x).astype(np.int8).\
#         add_prefix("lesion_1_").\
#         drop(columns = ['lesion_1_0', "lesion_1_6"], errors = "ignore");
        
#         df = pd.concat([df, l1_prf], axis=1);
#         del l1_prf;

#         if self.sec_ftre_req == "Y":
#             df['rectal_temp_risk'] = np.where(df.rectal_temp >= 37.8,1,0).astype(np.int8);
#             df['pulse_risk']       = np.where(df.pulse >= 40,1,0).astype(np.int8);
#             df['cell_vol_risk']    = np.where(df.packed_cell_volume >= 50, 1,0).astype(np.int8);
#             df['protein_risk']     = np.where(df.total_protein >= 7.5, 1,0).astype(np.int8);
                
#         if CFG.sec_ftre_req != "Y": 
#             PrintColor(f"Secondary features are not required", color = Fore.RED);    
        
#         self.op_cols = df.columns;  
#         return df;
    
#     def get_feature_names_in(self, X, y=None, **params): 
#         return self.ip_cols;    
    
#     def get_feature_names_out(self, X, y=None, **params): 
#         return self.op_cols;

# train_df['outcome'] = train_df['outcome'].map({"lived":1,"died":2,"euthanized":3}).astype(np.int8)
# original_df['outcome'] = original_df['outcome'].map({"lived":1,"died":2,"euthanized":3}).astype(np.int8)

class CatEncoder(TransformerMixin, BaseEstimator):
    def __init__(self): pass;
    
    def fit(self, X, y= None):
#         self.ip_cols = X.columns;
        return self
    
    def transform(self, X, y= None):
        df = X.copy()
        df['surgery'] = df['surgery'].map({'yes': 1, 'no': 2}).astype(np.int8)
        df['age']     = df['age'].map({'adult': 1, 'young': 2}).astype(np.int8)
        df['temp_of_extremities'] = df['temp_of_extremities'].map({'None': 0, "normal":1, "warm": 2, "cool":3, "cold":4}).astype(np.int8)
        df['peripheral_pulse'] = df['peripheral_pulse'].map({"None": 0, "normal":1, "increased": 2, "reduced":3, "absent":4}).astype(np.int8)
        df['mucous_membrane'] = df['mucous_membrane'].map({"None": 0,"normal_pink":1,"bright_pink":2, "pale_pink":3 , "pale_cyanotic": 4, "bright_red":5, "injected": 5, "dark_cyanotic": 6}).astype(np.int8);
        ## "3": 2, "more_3_sec": 2
        df['capillary_refill_time'] = df['capillary_refill_time'].map({"None": 0, "less_3_sec":1, "3": 2, "more_3_sec": 2}).astype(np.int8)
        ##'slight': 0, "moderate": 0需要改动
        df['pain'] = df['pain'].map({"None": 0, "alert" : 1, "no_pain": 1, "depressed": 2, "mild_pain": 3, 'slight': 0, "moderate": 0, "severe_pain": 4, "extreme_pain": 5}).astype(np.int8)
        ## 'distend_small':0
        df['peristalsis'] =df['peristalsis'].map({"None": 0, "hypermotile": 1, 'distend_small':0, "normal": 2,"hypomotile": 3, "absent": 4}).astype(np.int8)
        df['abdominal_distention'] = df['abdominal_distention'].map({"None": 0, "none": 1, "slight": 2, "moderate": 3, "severe": 4}).astype(np.int8)
        df['nasogastric_tube'] = df['nasogastric_tube'].map({"None": 0, "none": 1, "slight": 2, "significant": 3}).astype(np.int8)
        ## 'slight':0
        df['nasogastric_reflux'] = df['nasogastric_reflux'].map({"None": 0, "none": 1, 'slight':0, "less_1_liter": 2, "more_1_liter": 3}).astype(np.int8)
        ##  'serosanguious':0
        df['rectal_exam_feces'] = df['rectal_exam_feces'].map({"None": 0, "normal": 1, "increased": 2, "decreased": 3, "absent": 4, 'serosanguious':0}).astype(np.int8)
        df['abdomen'] = df['abdomen'].map({"None":0, "normal": 1, "other": 2, "firm": 3, "distend_small": 4, "distend_large": 5}).astype(np.int8)
        df['abdomo_appearance'] = df['abdomo_appearance'].map({"None":0, "clear": 1, "cloudy": 2, "serosanguious": 3}).astype(np.int8)
        df['surgical_lesion'] =df['surgical_lesion'].map({"yes": 1, "no": 0}).astype(np.int8)
        df['cp_data'] = df['cp_data'].map({"yes": 1, "no": 0}).astype(np.int8)
        
#         #  Encoding the lesion 2 as 0/ non-zero:-
#         df['lesion_2'] = df['lesion_2'].clip(0, 1).astype(np.int8)
        
        df[['hospital_number', 'lesion_1','lesion_2','lesion_3']] = df[['hospital_number', 'lesion_1','lesion_2','lesion_3']].astype(int)
#         self.op_cols = df.columns; 
        return df;
    
#     def get_feature_names_in(self, X, y=None): 
#         return self.ip_cols;    
    
#     def get_feature_names_out(self, X, y=None): 
#         return self.op_cols;



steps1 = [
         ("Imputer", ColumnTransformer([
                                         ("num_",
                                          Pipeline([('imputer', SimpleImputer(strategy="median")),('std_scaler', StandardScaler()),]),
                                          ['rectal_temp','pulse','respiratory_rate','nasogastric_reflux_ph','packed_cell_volume','total_protein','abdomo_protein']),
                                        ],
                                          remainder = SimpleImputer(strategy = "most_frequent"),verbose_feature_names_out = True
                                       )),
#          ('CatEncoder', CatEncoder()),
        ]
pipeline1 = Pipeline(steps = steps1, verbose = False)
display(pipeline1)

Xtrain, ytrain = train_df.drop(target, axis=1, errors = 'ignore'), train_df[target]
train_prepared1 = pipeline1.fit_transform(Xtrain, ytrain)
test_prepared1 = pipeline1.transform(test_df)
display(train_prepared1)
display(test_prepared1)

## Pipline后分析

In [ ]:
## 信息增益，互信息
def make_mi_scores(X, y):
    mi_scores = mutual_info_classif(X, y)
    columns = full_pipeline.get_feature_names_out()
    columns = np.delete(columns, len(columns) - 1)
    mi_scores = pd.Series(mi_scores, name="MI Scores", index=columns)
    mi_scores = mi_scores.sort_values(ascending=False)
    return mi_scores

X = train_prepared[:, :-1].copy()
y = train_prepared[:, -1]

mi_scores = make_mi_scores(X, y)
display(mi_scores[::3]) # show a few features with their MI scores
def plot_mi_scores(scores):
    scores = scores.sort_values(ascending=True)
    width = np.arange(len(scores))
    ticks = list(scores.index)
    plt.barh(width, scores)
    plt.yticks(width, ticks)
    plt.title("Mutual Information Scores")
plt.figure(dpi=100, figsize=(8, 5))
plot_mi_scores(mi_scores)

In [ ]:
sns.kdeplot(data=train_df, x='hospital_number', hue='outcome', fill=True)
plt.title("Distribution of hospital_number, by outcome")

## PCA分析

In [ ]:
train_X = train_prepared[:, :-1]
train_y = train_prepared[:, -1]
pca = PCA(n_components = 10)
X2D = pca.fit_transform(train_X)
display(pd.DataFrame(X2D))
display(pd.DataFrame(pca.explained_variance_ratio_))

## 网格搜索相关

In [ ]:
# # Pipeline specifics:-
# from sklearn.preprocessing import RobustScaler, MinMaxScaler, StandardScaler;
# from sklearn.impute import SimpleImputer as SI;
# from sklearn.model_selection import (RepeatedStratifiedKFold as RSKF, 
#                                      StratifiedKFold as SKF,
#                                      KFold, 
#                                      RepeatedKFold as RKF, 
#                                      cross_val_score);
# from sklearn.inspection import permutation_importance;
# from sklearn.feature_selection import mutual_info_classif, RFE;
# from sklearn.pipeline import Pipeline, make_pipeline;
# from sklearn.base import BaseEstimator, TransformerMixin;
# from sklearn.compose import ColumnTransformer;

# # ML Model training:-
# from sklearn.metrics import f1_score, confusion_matrix, make_scorer;
# from xgboost import DMatrix, XGBClassifier;
# from lightgbm import LGBMClassifier, log_evaluation, early_stopping;
# from catboost import CatBoostClassifier, Pool;
# from sklearn.ensemble import (RandomForestClassifier as RFC, 
#                               ExtraTreesClassifier as ETC,
#                               AdaBoostClassifier as ABC,
#                               BaggingClassifier as BC,
#                               HistGradientBoostingClassifier as HGBC
#                              );
# from sklearn.linear_model import LogisticRegression as LC;
# Mdl_Master = \
# {'CBC': CatBoostClassifier(**{'task_type'           : "GPU" if CFG.gpu_switch == "ON" else "CPU",
#                               'objective'           : "MultiClass",
#                               'eval_metric'         : "Accuracy",
#                               'classes_count'       : 3,
#                               'bagging_temperature' : 0.10,
#                               'colsample_bylevel'   : 0.75,
#                               'iterations'          : 1000,
#                               'learning_rate'       : 0.075,
#                               'od_wait'             : 3,
#                               'max_depth'           : 4,
#                               'l2_leaf_reg'         : 0.85,
#                               'min_data_in_leaf'    : 6,
#                               'random_strength'     : 0.65, 
#                               'max_bin'             : 80,
#                               'verbose'             : 0,
#                               'use_best_model'      : True,
#                            }
#                          ), 

#   'LGBMC': LGBMClassifier(**{'device'            : "gpu" if CFG.gpu_switch == "ON" else "cpu",
#                              'objective'         : 'multiclass',
#                              'metric'            : 'none',
#                              'boosting_type'     : 'gbdt',
#                              'random_state'      : CFG.state,
#                              'colsample_bytree'  : 0.5,
#                              'subsample'         : 0.65,
#                              'learning_rate'     : 0.08,
#                              'max_depth'         : 4,
#                              'n_estimators'      : 1000,
#                              'num_leaves'        : 72,                    
#                              'reg_alpha'         : 0.01,
#                              'reg_lambda'        : 1.75,
#                              'verbose'           : -1,
#                          }
#                       ),

#   'XGBC': XGBClassifier(**{'tree_method'        : "gpu_hist" if CFG.gpu_switch == "ON" else "hist",
#                            'objective'          : 'multi:softprob',
#                            'random_state'       : CFG.state,
#                            'colsample_bytree'   : 0.7,
#                            'learning_rate'      : 0.07,
#                            'max_depth'          : 4,
#                            'n_estimators'       : 1100,                         
#                            'reg_alpha'          : 0.025,
#                            'reg_lambda'         : 1.75,
#                            'min_child_weight'   : 5,
#                            'early_stopping_rounds' : CFG.nbrnd_erly_stp,
#                         }
#                        ),
 
#    'RFC' : RFC(n_estimators     = 150, 
#                criterion        = 'gini',
#                max_depth        = 4,
#                min_samples_leaf = 5,
#                max_features     = 'log2',
#                bootstrap        = True,
#                oob_score        = True,
#                random_state     = CFG.state,
#                verbose          =0,
#               ), 
 
#   "HGBC" : HGBC(loss              = 'categorical_crossentropy',
#                 learning_rate     = 0.075,
#                 early_stopping    = True,
#                 max_iter          = 200,
#                 max_depth         = 4,
#                 min_samples_leaf  = 5,
#                 l2_regularization = 1.75,
#                 scoring           = myscorer,
#                 random_state      = CFG.state,
#                )
# };

## 单模型----> 随机森林、LightGBM、XGBoost、CatBoost

## 均值融合、加权融合、Stacking融合(分类问题用逻辑回归、否则用贝叶斯回归(bayesianRidge))

In [ ]:
## trick